# Descarga del dataset

A continuación se procede a descargar el dataset y la explicación de sus atributos. Puedes ver la desscripción de las variables en el siguiente enlace:




In [5]:
#añade las librerías básicas que consideres necesarias para el proyecto (pongo 3). Posteriormente podrás añadir más librerías.
import os
import numpy as np
import pandas as pd

## Carga del fichero en Python
df_train = pd.read_csv('Leads.csv', delimiter=',')

df_train.head()


,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


# Enunciado

*  Construir un modelo de aprendizaje automático que
permita predecir si un cliente potencial se convertirá
en cliente real.

* Se trata de un problema de clasificación binaria. La variable objetivo será la columna Converted, que
indicará con un 1 si el cliente se ha convertido en
cliente real, o con un 0 en caso contrario.

* Se recomienda realizar un análisis del dataset y un preprocesamiento de los datos antes de generar el modelo. Puedes utilizar el modelo que desees.

* Las operaciones sobre el dataset se deberán encuadrar en cada una de las secciones que se definen a continuación en el documento.

* El modelo se ha conseguido entrenar con dicho dataset logrando una precisión del ~90%.


# 1. Análisis del dataset

Antes de comenzar con el preprocesamiento de los datos necesitamos analizarlos y entender un poco el dataset. Utiliza las diferentes funciones y gráficos que hemos visto para hacerte una idea del dataset con el que estamos trabajando. Esta información guiará el preprocesamiento de los datos.

# 2. Preprocesamiento del dataset


Dentro del preprocesamiento del dataset puedes utilizar diferentes técnicas como:**

*   Baleancear los datos
*   Eliminar columnas por los motivos que consideres (muchos valores nulos, no está correlacionado con la clase a predecir, ...)
*   Modificar columnas. Por ejemplo sutituir valores nulos por otros, clasificar una columna numérica en diferentes grupos (por ejemplo, cuartiles), ...
*   Crear nuevas columnas. Por ejemplo eliminando columnas categóricas, agregando un conjunto de columnas en otra, ...
*   Normalizar los datos
*   ...
*   

Al finalizar, deberás implementar un pipeline que paso a paso realice todo el preprocesamiento de los datos. De aquellos pasos que consideres necesarios, por ejemplo, el balanceamiento del dataset se deberá realizar fuera de un pipeline. 


# 3. Definición de modelos diferentes a redes neuronales

En este apartado podrás utilizar el algoritmo que consideres que ofrece mejores resultados para la generación del modelo. Para ello deberás realizar varios modelos y evaluarlos para quedarte con el que mejores resultados ofrezca.

# 4. Definición de modelo basado en red neuronal

En este apartado deberás entrenar una red neuronal ajustando la arquitectura y el entrenamiento a los mejores resultados de evaluación. 

# 5. Conclusiones

Se deberá realizar una explicación final del preprocesamiento de datos realizado, junto con el modelo que mejores resultados ha dado y cómo se ha evaluado dicho modelo. 